In [1]:
import os
import pathlib

ROOT = pathlib.Path().resolve().parent

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from datetime import datetime
import glob
from collections import Counter
import time
from collections import Counter
import urllib

nest_asyncio.apply()


## Подготовка данных для работы

In [91]:
import pathlib

data = pd.read_csv(ROOT / 'data/rambler.csv')
data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
print("Минимальная дата:", data['date'].min())
print("Максимальная дата:", data['date'].max())

Минимальная дата: 2004-01-01 00:00:00
Максимальная дата: 2020-10-31 00:00:00


In [7]:
from datetime import datetime as dt

dt_strt = data['date'].max()
dt_end = dt.now()

In [11]:
signs = ['aries', 'gemini', 'taurus', 'cancer', 'leo', 'virgo', 'libra', 'scorpio', 'sagittarius', 'capricorn',
         'aquarius', 'pisces']

In [26]:
list(pd.date_range(dt_strt, dt_end))

[Timestamp('2020-01-31 00:10:00'),
 Timestamp('2020-02-01 00:10:00'),
 Timestamp('2020-02-02 00:10:00'),
 Timestamp('2020-02-03 00:10:00'),
 Timestamp('2020-02-04 00:10:00'),
 Timestamp('2020-02-05 00:10:00'),
 Timestamp('2020-02-06 00:10:00'),
 Timestamp('2020-02-07 00:10:00'),
 Timestamp('2020-02-08 00:10:00'),
 Timestamp('2020-02-09 00:10:00'),
 Timestamp('2020-02-10 00:10:00'),
 Timestamp('2020-02-11 00:10:00'),
 Timestamp('2020-02-12 00:10:00'),
 Timestamp('2020-02-13 00:10:00'),
 Timestamp('2020-02-14 00:10:00'),
 Timestamp('2020-02-15 00:10:00'),
 Timestamp('2020-02-16 00:10:00'),
 Timestamp('2020-02-17 00:10:00'),
 Timestamp('2020-02-18 00:10:00'),
 Timestamp('2020-02-19 00:10:00'),
 Timestamp('2020-02-20 00:10:00'),
 Timestamp('2020-02-21 00:10:00'),
 Timestamp('2020-02-22 00:10:00'),
 Timestamp('2020-02-23 00:10:00'),
 Timestamp('2020-02-24 00:10:00'),
 Timestamp('2020-02-25 00:10:00'),
 Timestamp('2020-02-26 00:10:00'),
 Timestamp('2020-02-27 00:10:00'),
 Timestamp('2020-02-

## Парсинг рамблера

In [9]:
year_strt = dt_strt.year
year_end = dt_end.year

print('Начало периода:', year_strt)
print('Конец периода:', year_end)

Начало периода: 2020
Конец периода: 2023


In [46]:
new = pd.DataFrame()

for i in range(year_strt, year_end):
    for j in range(1, 13):
        for k in range(1, 32):
            for sign in signs:
                try:
                    year = i
                    month = j
                    day = k
                    if month < 10:
                        month = '0' + str(month)
                    if day < 10:
                        day = '0' + str(day)
                    date = '%s-%s-%s' % (year, month, day)
                    url = 'https://horoscopes.rambler.ru/%s/%s/?updated' % (sign, date)  # url для второй страницы
                    print(url)
                    r = requests.get(url)
                    response = r.text.encode('utf-8')

                    soup = BeautifulSoup(response, features="lxml")
                    text = soup.find('div', {'class': '_1E4Zo'}).text

                    temp = pd.DataFrame({'date': date, 'sign': soup, 'text': text}, index=[0])
                    new = pd.concat([new, temp])
                    print('saved date=', date, ' sign=', sign)

                except Exception:
                    continue

https://horoscopes.rambler.ru/aries/2020-01-01/?updated
saved date= 2020-01-01  sign= aries
https://horoscopes.rambler.ru/gemini/2020-01-01/?updated
saved date= 2020-01-01  sign= gemini
https://horoscopes.rambler.ru/taurus/2020-01-01/?updated
saved date= 2020-01-01  sign= taurus
https://horoscopes.rambler.ru/cancer/2020-01-01/?updated
saved date= 2020-01-01  sign= cancer
https://horoscopes.rambler.ru/leo/2020-01-01/?updated
saved date= 2020-01-01  sign= leo
https://horoscopes.rambler.ru/virgo/2020-01-01/?updated
saved date= 2020-01-01  sign= virgo
https://horoscopes.rambler.ru/libra/2020-01-01/?updated
saved date= 2020-01-01  sign= libra
https://horoscopes.rambler.ru/scorpio/2020-01-01/?updated
saved date= 2020-01-01  sign= scorpio
https://horoscopes.rambler.ru/sagittarius/2020-01-01/?updated
saved date= 2020-01-01  sign= sagittarius
https://horoscopes.rambler.ru/capricorn/2020-01-01/?updated
saved date= 2020-01-01  sign= capricorn
https://horoscopes.rambler.ru/aquarius/2020-01-01/?upd

In [66]:
# fix with sign
signs_v = signs * (len(new) // 12)

new['sign'] = signs_v

new

,date,sign,text
0,2020-01-02,aries,В первые сутки нового календарного года Овнам ...
0,2020-01-02,gemini,Сегодня звезды советуют Близнецам придерживать...
0,2020-01-02,taurus,"На Тельцов сегодня скрыто, но мощно воздейству..."
0,2020-01-02,cancer,Ракам первый январский день поможет кристаллиз...
0,2020-01-02,leo,В первый день календарного года Львам противоп...
...,...,...,...
0,2020-01-02,scorpio,Сегодня Скорпионов поглотит новогодняя суета –...
0,2020-01-02,sagittarius,"Сегодня у Стрельцов не все идет по плану, и, в..."
0,2020-01-02,capricorn,Сегодня Козерогам стоит быть во всем осторожне...
0,2020-01-02,aquarius,Сегодня Водолеям придется подстраиваться под о...


In [78]:
time_range = pd.date_range(start='2020-01-02', end='2023-01-01', freq='1D')

for sign in signs:
    new.loc[new["sign"] == sign,'date'] = time_range

In [92]:
new

,date,sign,text
0,2020-01-02 00:00:00,aries,В первые сутки нового календарного года Овнам ...
0,2020-01-02 00:00:00,gemini,Сегодня звезды советуют Близнецам придерживать...
0,2020-01-02 00:00:00,taurus,"На Тельцов сегодня скрыто, но мощно воздейству..."
0,2020-01-02 00:00:00,cancer,Ракам первый январский день поможет кристаллиз...
0,2020-01-02 00:00:00,leo,В первый день календарного года Львам противоп...
...,...,...,...
0,2023-01-01 00:00:00,scorpio,Сегодня Скорпионов поглотит новогодняя суета –...
0,2023-01-01 00:00:00,sagittarius,"Сегодня у Стрельцов не все идет по плану, и, в..."
0,2023-01-01 00:00:00,capricorn,Сегодня Козерогам стоит быть во всем осторожне...
0,2023-01-01 00:00:00,aquarius,Сегодня Водолеям придется подстраиваться под о...


In [107]:
dataset = pd.concat([data, new])

dataset = dataset.drop_duplicates(subset=['date', 'sign'])
dataset

,date,sign,text
0,2004-01-01 00:00:00,aries,"Любые разногласия во мнениях скоро улягутся, а..."
1,2004-01-01 00:00:00,gemini,Первый день нового года прекрасно подходит для...
2,2004-01-01 00:00:00,taurus,Этот день как нельзя лучше подходит для улучше...
3,2004-01-01 00:00:00,cancer,"Первый день нового года, да еще понедельник бу..."
4,2004-01-01 00:00:00,leo,Львы не сорвутся в начале года с места в карье...
...,...,...,...
0,2023-01-01 00:00:00,scorpio,Сегодня Скорпионов поглотит новогодняя суета –...
0,2023-01-01 00:00:00,sagittarius,"Сегодня у Стрельцов не все идет по плану, и, в..."
0,2023-01-01 00:00:00,capricorn,Сегодня Козерогам стоит быть во всем осторожне...
0,2023-01-01 00:00:00,aquarius,Сегодня Водолеям придется подстраиваться под о...


In [108]:
#dataset = dataset.drop_duplicates()

dataset = dataset.reset_index(drop=True)

In [109]:
dataset.to_csv(ROOT / 'data/rambler.csv', sep='\t', index=False)

## Сайт Ростова-на-Дону

https://www.1rnd.ru/

In [ ]:
news = []
url = 'https://www.1rnd.ru/news/3089599'
print(url)
r = requests.get(url)
response = r.text
soup = BeautifulSoup(response, 'html.parser')

news_text = soup.findAll('p', {'style': "text-align:justify;"})
date_text = soup.findAll('div', {'class': "article-info__time"})
header_text = soup.findAll('div', {'class': 'title-container inner-title'})

if news_text:
    print('success')
    text_line = ''
    for text in news_text:
        text_line = text_line + text.get_text()
    news.append([header_text[0].get_text(), date_text[0].get_text().split(), text_line])

In [ ]:
news = pd.DataFrame({'header': [0], 'date': [0], 'news': [0]})

In [ ]:
start1 = 7135
start2 = 300000
start3 = 600000
start4 = 900000
start5 = 1200000
start6 = 1500000
start7 = 1800000
start8 = 2100000
start9 = 2400000
start10 = 2700000

end1 = 299999
end2 = 599999
end3 = 899999
end4 = 1199999
end5 = 1499999
end6 = 1799999
end7 = 2099999
end8 = 2399999
end9 = 2699999
end10 = 2999999


async def get_news(start, end):
    global news
    for i in range(start, end):
        url = 'https://www.1rnd.ru/news/{0}'.format(str(i))
        print(url)

        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:

                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                news_text = soup.findAll('p', {'style': "text-align:justify;"})
                date_text = soup.findAll('div', {'class': "article-info__time"})
                header_text = soup.findAll('div', {'class': 'title-container inner-title'})

                if news_text:
                    print('success')
                    text_line = ''
                    for text in news_text:
                        text_line = text_line + text.get_text()

                    date = date_text[0].get_text()
                    header = header_text[0].get_text()

                    news = news.append({'header': header, 'date': date, 'news': text_line}, ignore_index=True)
                    news.to_csv('news.csv', encoding='utf-8')


futures = [get_news(start1, end1),
           get_news(start2, end2),
           get_news(start3, end3),
           get_news(start4, end4),
           get_news(start5, end5),
           get_news(start6, end6),
           get_news(start7, end7),
           get_news(start8, end8),
           get_news(start9, end9),
           get_news(start10, end10)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
loop.close()